Initiate Spark Session, insert dataset and create initial RDD

In [12]:
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
pagecounts = sc.textFile("pagecounts.out")

Equivalent to case class of scala in python (also tried dataclass or simple class but their output caused a PicklingError in the lambda function later)

In [14]:
from dataclasses import dataclass

@dataclass
class Log2:
    ProjectCode: str
    PageTitle: str
    PageHits: str
    PageSize: str
        
line = Log2("a","b","c","d")
print(line)

x="a s f b"
def conv(x):
    return Log2(splitter(x)[0],splitter(x)[1],splitter(x)[2],splitter(x)[3])
conv(x)
#pagecounts.map(lambda x: conv(x)).take(5)

Log2(ProjectCode='a', PageTitle='b', PageHits='c', PageSize='d')


Log2(ProjectCode='a', PageTitle='s', PageHits='f', PageSize='b')

In [13]:
from collections import namedtuple

Log = namedtuple("Log", ["ProjectCode", "PageTitle", "PageHits", "PageSize"])

Function which splits lines into words

In [15]:
def splitter(line):
    return line.split(" ")

Function getting RDD[STRING] as input and returning RDD[LOG] 

In [16]:
def content(RDD):
    return RDD.map(lambda line: Log(splitter(line)[0],splitter(line)[1],splitter(line)[2],splitter(line)[3]))

Question 1. Retrieve the first 15 records and print out the result.

In [49]:
LOG=content(pagecounts)
LOG.take(15)

[Log(ProjectCode='aa', PageTitle='271_a.C', PageHits='1', PageSize='4675'),
 Log(ProjectCode='aa', PageTitle='Category:User_th', PageHits='1', PageSize='4770'),
 Log(ProjectCode='aa', PageTitle='Chiron_Elias_Krase', PageHits='1', PageSize='4694'),
 Log(ProjectCode='aa', PageTitle='Dassault_rafaele', PageHits='2', PageSize='9372'),
 Log(ProjectCode='aa', PageTitle='E.Desv', PageHits='1', PageSize='4662'),
 Log(ProjectCode='aa', PageTitle='File:Wiktionary-logo-en.png', PageHits='1', PageSize='10752'),
 Log(ProjectCode='aa', PageTitle='Indonesian_Wikipedia', PageHits='1', PageSize='4679'),
 Log(ProjectCode='aa', PageTitle='Main_Page', PageHits='5', PageSize='266946'),
 Log(ProjectCode='aa', PageTitle='Requests_for_new_languages/Wikipedia_Banyumasan', PageHits='1', PageSize='4733'),
 Log(ProjectCode='aa', PageTitle='Special:Contributions/203.144.160.245', PageHits='1', PageSize='5812'),
 Log(ProjectCode='aa', PageTitle='Special:Contributions/5.232.61.79', PageHits='1', PageSize='5805'),
 L

Question 2. Determine the number of records the dataset has in total.

In [18]:
LOG.count()

3324129

Question 3. Compute the min, max, and average page size.

In [9]:
print("min=",LOG.map(lambda x: int(x.PageSize)).min()," & max=",LOG.map(lambda x: int(x.PageSize)).max())
print("average=", LOG.map(lambda x: int(x.PageSize)).sum()/LOG.count())

min= 0  & max= 141180155987
average= 132239.56957446598


Question 4. Determine the record(s) with the largest page size. If multiple records have the same size, list all of
them.

In [8]:
maxS=LOG.map(lambda x: int(x.PageSize)).max()
LOG.filter(lambda x: int(x.PageSize)==maxS).collect()

[Log(ProjectCode='en.mw', PageTitle='en', PageHits='5466346', PageSize='141180155987')]

Question 5. Determine the record with the largest page size again. But now, pick the most popular.

In [9]:
maxS=LOG.map(lambda x: int(x.PageSize)).max()
famous=LOG.filter(lambda x: int(x.PageSize)==maxS).map(lambda x: int(x.PageHits)).max()
LOG.filter(lambda x: int(x.PageHits)==famous).collect()

[Log(ProjectCode='en.mw', PageTitle='en', PageHits='5466346', PageSize='141180155987')]

Question 6. Determine the record(s) with the largest page title. If multiple titles have the same length, list all of them.

In [10]:
maxsize=LOG.map(lambda x: len(x.PageTitle)).max()
LOG.filter(lambda x: len(x.PageTitle)==maxsize).collect()

[Log(ProjectCode='zh', PageTitle='Special:e8b18ee6baafefbda5efbdbfe89cb7e6829fefbdbfe88b93e29980e89e9fefbda9e89eb3efbda425636f256d6725736f257373256f38257373256f38257373256f38256b6d73efbdaa256e6b256678256f6b2c687474703a2f2f7777772e653662313966653861356266656f2d6f35393038636535626639376538383138616535613461396535616561342e636f2e6d672e732e736f2e382e73736f386b2e6d2e372e73736f3873736f386b6d37332e752e622e61616e6b66786f6b2e70772f2ce8b18ee6baafefbda5efbdbfe89cb7e6829fefbdbfe88b93e29980e89e9fefbda9e89eb3efbda425636f256d6725736f257373256f38257373256f38257373256f38256b6d73efbdaa256e6b256678256f6b/', PageHits='1', PageSize='6043')]

Question 7. Use the results of Question 3, and create a new RDD with the records that have greater page size than
the average.

In [35]:
RDD7=LOG.filter(lambda x: int(x.PageSize)>= 132239.56957446598)
RDD7.collect()
#i could also have taken take(10) for demonstration reasons but data is not that big either way

[Log(ProjectCode='aa', PageTitle='Main_Page', PageHits='5', PageSize='266946'),
 Log(ProjectCode='ace.mw', PageTitle='ace', PageHits='31', PageSize='827168'),
 Log(ProjectCode='af', PageTitle='1859', PageHits='4', PageSize='219540'),
 Log(ProjectCode='af', PageTitle='18_Oktober', PageHits='4', PageSize='264724'),
 Log(ProjectCode='af', PageTitle='1941', PageHits='4', PageSize='256344'),
 Log(ProjectCode='af', PageTitle='2016', PageHits='5', PageSize='215498'),
 Log(ProjectCode='af', PageTitle='4_Januarie', PageHits='4', PageSize='268828'),
 Log(ProjectCode='af', PageTitle='Afrika-unie', PageHits='1', PageSize='172078'),
 Log(ProjectCode='af', PageTitle='Big_Ben', PageHits='13', PageSize='136201'),
 Log(ProjectCode='af', PageTitle='Comrades-maraton', PageHits='1', PageSize='155180'),
 Log(ProjectCode='af', PageTitle='Dmitri_Medwedef', PageHits='2', PageSize='141328'),
 Log(ProjectCode='af', PageTitle='Elsas', PageHits='4', PageSize='319408'),
 Log(ProjectCode='af', PageTitle='Engels', P

Question 8. Compute the total number of pageviews for each project (as the schema shows, the first field of each
record contains the project code).

In [14]:
LOG.map(lambda x: (x.ProjectCode,x.PageHits)).reduceByKey(lambda x, y: int(x)+int(y)).collect()

[('aa', 41),
 ('ab.mw', '6'),
 ('af', 2494),
 ('af.d', 123),
 ('af.mw', '532'),
 ('af.q', 30),
 ('ak.b', 3),
 ('an', 799),
 ('an.v', '1'),
 ('ar', 1835),
 ('ar.d', 234),
 ('ar.mw', '118349'),
 ('arbcom-en', 36),
 ('arc.mw', '8'),
 ('arz.mw', '2271'),
 ('as.mw', '26'),
 ('ast.b', 5),
 ('ay.d', 45),
 ('az', 1487),
 ('az.q', 9),
 ('azb.v', '1'),
 ('bar', 804),
 ('be-tarask.mw', '33'),
 ('be.b', '2'),
 ('bi.b', 25),
 ('bm.d', 2),
 ('bn', 112),
 ('bo.d', 4),
 ('br', 1606),
 ('br.v', '1'),
 ('bs.mw', '2523'),
 ('bs.n', 37),
 ('bs.v', 2),
 ('bxr', 12),
 ('ca', 11341),
 ('cbk-zam.mw', '3'),
 ('ce.mw', '37'),
 ('chy', 62),
 ('co.d', 31),
 ('co.mw', '37'),
 ('crh.mw', '9'),
 ('cs', 19881),
 ('cs.b', 22),
 ('cs.n', 40),
 ('cu.mw', '16'),
 ('cy', 1762),
 ('cy.mw', '310'),
 ('cy.s', 21),
 ('de.d', 5324),
 ('diq', 268),
 ('dsb', 135),
 ('dsb.mw', '36'),
 ('el', 1016),
 ('eml', 166),
 ('en.mw', '5466346'),
 ('en.n', 16765),
 ('eo.d', 367),
 ('eo.n', 31),
 ('es.v', 116),
 ('es.voy', 84),
 ('et.v', '1'

In [25]:
projectviews=LOG.map(lambda x: (x.ProjectCode,x.PageHits)).reduceByKey(lambda x, y: int(x)+int(y))
projectviews.collect()

[('aa', 41),
 ('ab.mw', '6'),
 ('af', 2494),
 ('af.d', 123),
 ('af.mw', '532'),
 ('af.q', 30),
 ('ak.b', 3),
 ('an', 799),
 ('an.v', '1'),
 ('ar', 1835),
 ('ar.d', 234),
 ('ar.mw', '118349'),
 ('arbcom-en', 36),
 ('arc.mw', '8'),
 ('arz.mw', '2271'),
 ('as.mw', '26'),
 ('ast.b', 5),
 ('ay.d', 45),
 ('az', 1487),
 ('az.q', 9),
 ('azb.v', '1'),
 ('bar', 804),
 ('be-tarask.mw', '33'),
 ('be.b', '2'),
 ('bi.b', 25),
 ('bm.d', 2),
 ('bn', 112),
 ('bo.d', 4),
 ('br', 1606),
 ('br.v', '1'),
 ('bs.mw', '2523'),
 ('bs.n', 37),
 ('bs.v', 2),
 ('bxr', 12),
 ('ca', 11341),
 ('cbk-zam.mw', '3'),
 ('ce.mw', '37'),
 ('chy', 62),
 ('co.d', 31),
 ('co.mw', '37'),
 ('crh.mw', '9'),
 ('cs', 19881),
 ('cs.b', 22),
 ('cs.n', 40),
 ('cu.mw', '16'),
 ('cy', 1762),
 ('cy.mw', '310'),
 ('cy.s', 21),
 ('de.d', 5324),
 ('diq', 268),
 ('dsb', 135),
 ('dsb.mw', '36'),
 ('el', 1016),
 ('eml', 166),
 ('en.mw', '5466346'),
 ('en.n', 16765),
 ('eo.d', 367),
 ('eo.n', 31),
 ('es.v', 116),
 ('es.voy', 84),
 ('et.v', '1'

Question 9. Report the 10 most popular pageviews of all projects, sorted by the total number of hits.

In [46]:
LOG.map(lambda x: (x.ProjectCode,x.PageHits)).reduceByKey(lambda x, y: int(x)+int(y)) \
        .takeOrdered(10, lambda pair: -int(pair[1]))

[('en.mw', '5466346'),
 ('en', 4959090),
 ('es.mw', '695531'),
 ('ja.mw', '611443'),
 ('de.mw', '572119'),
 ('fr.mw', '536978'),
 ('ru.mw', '466742'),
 ('it.mw', '400297'),
 ('de', 315929),
 ('commons.m', 285796)]

In [26]:
projectviews.takeOrdered(10, lambda pair: -int(pair[1]))

[('en.mw', '5466346'),
 ('en', 4959090),
 ('es.mw', '695531'),
 ('ja.mw', '611443'),
 ('de.mw', '572119'),
 ('fr.mw', '536978'),
 ('ru.mw', '466742'),
 ('it.mw', '400297'),
 ('de', 315929),
 ('commons.m', 285796)]

Question 10. Determine the number of page titles that start with the article “The”. How many of those page titles
are not part of the English project (Pages that are part of the English project have “en” as the first
field)?

In [17]:
LOG.filter(lambda x: x.PageTitle.startswith("The")).count()

45020

In [27]:
thepages=LOG.filter(lambda x: x.PageTitle.startswith("The"))
thepages.count()

45020

In [18]:
LOG.filter(lambda x: x.PageTitle.startswith("The")).filter(lambda x: x.ProjectCode!="en").count()

10292

In [28]:
thepages.filter(lambda x: x.ProjectCode!="en").count()

10292

Question 11. Determine the percentage of pages that have only received a single page view in this one hour of log
data.

In [19]:
LOG.filter(lambda x: x.PageHits=="1").count()/LOG.count()

0.7696247648632168

Question 12. Determine the number of unique terms appearing in the page titles. Note that in page titles, terms
are delimited by “ ” instead of a white space. You can use any number of normalization steps (e.g.,
lowercasing, removal of non-alphanumeric characters).

In [63]:
import re
LOG.flatMap(lambda line: line.PageTitle.lower().split("_")).map(lambda el:re.sub(r'\W', '', el)).distinct().count()

1689812

Question 13. Determine the most frequently occurring page title term in this dataset.

In [24]:
freq=LOG.map(lambda line: (line.PageTitle,1)).reduceByKey(lambda x, y: x+y).values().max()
LOG.map(lambda line: (line.PageTitle,1)).reduceByKey(lambda x, y: x+y) \
    .filter(lambda value: value[1]==freq).collect()

[('water', 118)]

In [29]:
titlesfreq=LOG.map(lambda line: (line.PageTitle,1)).reduceByKey(lambda x, y: x+y)
freq=titlesfreq.values().max()
titlesfreq.filter(lambda value: value[1]==freq).collect()

[('water', 118)]

# Task 2

In [19]:
from pyspark.sql.types import IntegerType, LongType 
df=LOG.toDF()
df = df.withColumn("PageHits", df["PageHits"].cast(IntegerType()))
df = df.withColumn("PageSize", df["PageSize"].cast(LongType()))
#I initially had converted both into integers bu I was missing 11 values which were long
df.printSchema()
df.show(5)

root
 |-- ProjectCode: string (nullable = true)
 |-- PageTitle: string (nullable = true)
 |-- PageHits: integer (nullable = true)
 |-- PageSize: long (nullable = true)

+-----------+------------------+--------+--------+
|ProjectCode|         PageTitle|PageHits|PageSize|
+-----------+------------------+--------+--------+
|         aa|           271_a.C|       1|    4675|
|         aa|  Category:User_th|       1|    4770|
|         aa|Chiron_Elias_Krase|       1|    4694|
|         aa|  Dassault_rafaele|       2|    9372|
|         aa|            E.Desv|       1|    4662|
+-----------+------------------+--------+--------+
only showing top 5 rows



Question 3. Compute the min, max, and average page size.

In [20]:
from pyspark.sql import functions as F 
df.select(F.min("PageSize"), F.max("PageSize"), F.avg("PageSize")).show()

+-------------+-------------+------------------+
|min(PageSize)|max(PageSize)|     avg(PageSize)|
+-------------+-------------+------------------+
|            0| 141180155987|132239.56957446598|
+-------------+-------------+------------------+



Question 5. Determine the record with the largest page size again. But now, pick the most popular.

In [51]:
maxS=df.select(F.max("PageSize")).first()[0]  #first: return Row
famous=df.filter(df.PageSize==maxS).select(F.max("PageHits")).first()[0]
df.filter(df.PageHits==famous).show()


+-----------+---------+--------+------------+
|ProjectCode|PageTitle|PageHits|    PageSize|
+-----------+---------+--------+------------+
|      en.mw|       en| 5466346|141180155987|
+-----------+---------+--------+------------+



Question 7. Use the results of Question 3, and create a new RDD with the records that have greater page size than
the average.

In [34]:
larger=df.filter(df.PageSize>=132239.56957446598).show()

+-----------+--------------------+--------+--------+
|ProjectCode|           PageTitle|PageHits|PageSize|
+-----------+--------------------+--------+--------+
|         aa|           Main_Page|       5|  266946|
|     ace.mw|                 ace|      31|  827168|
|         af|                1859|       4|  219540|
|         af|          18_Oktober|       4|  264724|
|         af|                1941|       4|  256344|
|         af|                2016|       5|  215498|
|         af|          4_Januarie|       4|  268828|
|         af|         Afrika-unie|       1|  172078|
|         af|             Big_Ben|      13|  136201|
|         af|    Comrades-maraton|       1|  155180|
|         af|     Dmitri_Medwedef|       2|  141328|
|         af|               Elsas|       4|  319408|
|         af|              Engels|       2|  182375|
|         af|         Erich_Fromm|       4|  215612|
|         af|            Filosoof|       2|  134400|
|         af|                GNTA|      77|  5

Question 12. Determine the number of unique terms appearing in the page titles. Note that in page titles, terms
are delimited by “ ” instead of a white space. You can use any number of normalization steps (e.g.,
lowercasing, removal of non-alphanumeric characters).

In [56]:
#from pyspark.sql.functions import split, countDistinct, array_distinct, size
split_col = df.select(F.split('PageTitle',"_").alias("Split")) #splits the words and puts them into an array
split2 = split_col.select(split_col.Split,F.explode(split_col.Split))\
                    .withColumn("col", F.regexp_replace("col",'\W', ''))

#new DF with columns array, element of array
#              Split  col
#eg [a,b,c]-->[a,b,c] a
#             [a,b,c] b
#             [a,b,c] c  

split3=split2.select(F.lower("col"))
split3.show(6)

count=split3.select(F.countDistinct("lower(col)"))
count.show()


+------------+
|  lower(col)|
+------------+
|         271|
|          ac|
|categoryuser|
|          th|
|      chiron|
|       elias|
+------------+
only showing top 6 rows

+--------------------------+
|count(DISTINCT lower(col))|
+--------------------------+
|                   1688529|
+--------------------------+



Question 13. Determine the most frequently occurring page title term in this dataset.

In [80]:
maxocc = df.groupBy("PageTitle").agg(F.count("PageTitle").alias("occurences"))
number = maxocc.select(F.max("occurences")).first()[0]
maxocc.filter(maxocc.occurences==number).show()

+---------+----------+
|PageTitle|occurences|
+---------+----------+
|    water|       118|
+---------+----------+

